Criar uma tabela de Clientes para equipe de marketing:
Importante somente pessoas que tenham e-mail e telefone cadastrados

In [0]:
%python
bronze_path   = 'abfss://uc-ext-azure@externalcursodb.dfs.core.windows.net/bikestore/bronze/'
silver_path   = 'abfss://uc-ext-azure@externalcursodb.dfs.core.windows.net/bikestore/silver/'
gold_path     = 'abfss://uc-ext-azure@externalcursodb.dfs.core.windows.net/bikestore/gold/'
resource_path = 'abfss://uc-ext-azure@externalcursodb.dfs.core.windows.net/bikestore/resource/origem/'
resource_path_volume = '/Volumes/infinitybigdata/azure/bikestore_resource/origem/'

In [0]:
%python
# criar varias tabelas temporárias de forma prática
bronze_map = {
    #"tmp_bronze_brands":      f"{bronze_path}/brands/",
    #"tmp_bronze_categories":  f"{bronze_path}/categories/",
    "tmp_bronze_customers":   f"{bronze_path}/customers/",
    #"tmp_bronze_order_items": f"{bronze_path}/order_items/",
    #"tmp_bronze_orders":      f"{bronze_path}/orders/",
    #"tmp_bronze_products":    f"{bronze_path}/products/",
    #"tmp_bronze_staffs":      f"{bronze_path}/staffs/",
    #"tmp_bronze_stocks":      f"{bronze_path}/stocks/",
    #"tmp_bronze_stores":      f"{bronze_path}/stores/",
}
for view_name, path in bronze_map.items():
    (spark.read.format('delta')
        .load(path)
        .createOrReplaceTempView(view_name))

In [0]:
SELECT *
FROM tmp_bronze_customers
WHERE 

In [0]:
DESCRIBE tmp_bronze_customers

In [0]:
SELECT 
CT.customer_id,
CT.first_name,
CT.last_name,
CT.phone,
CT.email,
CT.street,
CT.city
FROM tmp_bronze_customers AS CT
WHERE 1=1 --só pra organizar, não influencia em nada no código
AND CT.email IS NOT NULL 
AND CT.email NOT IN ('NULL', 'NULL ')
AND CT.phone IS NOT NULL 
AND CT.phone NOT IN ('NULL', 'NULL ')


In [0]:
%python
df_customers_silver = spark.sql(
    """
    SELECT 
      CT.customer_id,
      CT.first_name,
      CT.last_name,
      CT.phone,
      CT.email,
      CT.street,
      CT.city
    FROM tmp_bronze_customers AS CT
    WHERE 1=1 --só pra organizar, não influencia em nada no código
      AND CT.email IS NOT NULL 
      AND CT.email NOT IN ('NULL', 'NULL ')
      AND CT.phone IS NOT NULL 
      AND CT.phone NOT IN ('NULL', 'NULL ')
"""
)

# salva em delta
df_customers_silver.write.mode("overwrite").format("delta").option(
    "mergeSchema", "true"
).save(f"{silver_path}/customers")

In [0]:
CREATE TABLE bikestore.logistics.customers_silver
AS
SELECT *
FROM delta.`abfss://uc-ext-azure@externalcursodb.dfs.core.windows.net/bikestore/silver/customers`;

In [0]:
SELECT *
FROM bikestore.logistics.customers_silver